# City wide context analysis of urban change categories
This notebook compares the spatial distribution of four key urban change categories across three urban zones: slums, 100m buffer around slums, and other city regions.

**Author:** Sai Ganesh Veeravalli  
**Dataset Used:** Google 2.5D Open Buildings (Temporal), Nairobi IDEAtlas Slum reference data, Nairobi Administrative boundary from HDX  

# 📦 Section 1: Import Required Libraries

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np

## Section 1.1 Load and preprocess spatial layers

In [ ]:
# Load classified urban change grids (2016–2023)
grid = gpd.read_file(r"F:\DEPRIMAP\EARSEL2025\March2025\Data\csv_experiment\urban_change_updated_23_16.gpkg")

# Load slum polygons and city boundary
slums = gpd.read_file(r"F:\DEPRIMAP\EARSEL2025\DATASETS\Reference Slum Parcels\nairobi_reference_slums.shp")
nairobi_city = gpd.read_file(r"F:\DEPRIMAP\EARSEL2025\DATASETS\AdminBoundaries\Nairobi_admin_boundary.shp")

# Reproject all layers to EPSG:3857 (for accurate buffering and area computation)
grid = grid.to_crs(epsg=3857)
slums = slums.to_crs(epsg=3857)
nairobi_city = nairobi_city.to_crs(epsg=3857)

# Fix any minor geometry issues (e.g., self-intersections)
grid['geometry'] = grid['geometry'].buffer(0)
slums['geometry'] = slums['geometry'].buffer(0)
nairobi_city['geometry'] = nairobi_city['geometry'].buffer(0)

# 📈  Section 2: City context analysis 

## Section 2.1 Derive spatial zones

In [ ]:
# Step 1: Dissolve Nairobi city boundary to a single geometry
nairobi_city_dissolved = nairobi_city.dissolve()

# Step 2: Clip slums to fit within city boundary
slums_clipped = gpd.overlay(slums, nairobi_city_dissolved, how='intersection')

# Step 3: Create a 100m buffer around clipped slums
slums_buffered = slums_clipped.copy()
slums_buffered['geometry'] = slums_buffered.geometry.buffer(100)

# Step 4: Dissolve buffer and clip it to city boundary
buffer_dissolved = slums_buffered.dissolve()
buffer_clipped = gpd.overlay(buffer_dissolved, nairobi_city_dissolved, how='intersection')

# Step 5: Dissolve slums for clean subtraction
slums_dissolved = slums_clipped.dissolve()

# Step 6: Subtract slums from buffer → get 100m buffer ring only
buffer_ring_geom = buffer_clipped.geometry.difference(slums_dissolved.geometry)
buffer_ring_gdf = gpd.GeoDataFrame(geometry=buffer_ring_geom, crs=slums.crs).reset_index(drop=True)

# Step 7: Subtract slums and buffer from city → get "other" regions
city_minus_slums = nairobi_city_dissolved.geometry.difference(slums_dissolved.geometry)
other_geom = city_minus_slums.difference(buffer_clipped.geometry)
other_regions_gdf = gpd.GeoDataFrame(geometry=other_geom, crs=slums.crs).reset_index(drop=True)

## Section 2.2 Visualize the three spatial zones

In [ ]:
fig, ax = plt.subplots(figsize=(12, 12))

nairobi_city_dissolved.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=1.2, label='City Boundary')
other_regions_gdf.plot(ax=ax, facecolor='lightgreen', edgecolor='green', alpha=0.4, label='Other Regions')
buffer_ring_gdf.plot(ax=ax, facecolor='none', edgecolor='red', linestyle='--', linewidth=1.2, label='Buffer Ring')
slums_clipped.plot(ax=ax, facecolor='lightgray', edgecolor='blue', linewidth=0.8, label='Slums')

ax.set_title("Clipped Spatial Zones Within Nairobi City", fontsize=14)
ax.set_aspect('equal')
ax.legend(loc='upper right')
plt.axis('off')
plt.show()

## Section 2.3 Spatial overlay with grid and filter by 4 focus categories

In [ ]:
target_categories = [
    'Vertical Densification',
    'High Densification',
    'Horizontal Densification',
    'Decline'
]

In [ ]:
# Slum grids
grids_in_slums = gpd.overlay(grid, slums_clipped, how='intersection')
grids_in_slums = grids_in_slums[grids_in_slums['urban_change'].isin(target_categories)].copy()
grids_in_slums['area_sqm'] = grids_in_slums.geometry.area
grids_in_slums['area_ha'] = grids_in_slums['area_sqm'] / 10_000

# Buffer ring grids
grids_in_buffer = gpd.overlay(grid, buffer_ring_gdf, how='intersection')
grids_in_buffer = grids_in_buffer[grids_in_buffer['urban_change'].isin(target_categories)].copy()
grids_in_buffer['area_sqm'] = grids_in_buffer.geometry.area
grids_in_buffer['area_ha'] = grids_in_buffer['area_sqm'] / 10_000

# Other region grids
grids_in_other = gpd.overlay(grid, other_regions_gdf, how='intersection')
grids_in_other = grids_in_other[grids_in_other['urban_change'].isin(target_categories)].copy()
grids_in_other['area_sqm'] = grids_in_other.geometry.area
grids_in_other['area_ha'] = grids_in_other['area_sqm'] / 10_000

## Section 2.4 Aggregate area statistics per zone

In [ ]:
# Summarize total area (in hectares) by category and zone
slum_summary = grids_in_slums.groupby('urban_change')['area_ha'].sum().reset_index().rename(columns={'area_ha': 'slum_area_ha'})
buffer_summary = grids_in_buffer.groupby('urban_change')['area_ha'].sum().reset_index().rename(columns={'area_ha': 'buffer_area_ha'})
other_summary = grids_in_other.groupby('urban_change')['area_ha'].sum().reset_index().rename(columns={'area_ha': 'other_area_ha'})

# Merge all summaries
area_summary = slum_summary.merge(buffer_summary, on='urban_change', how='outer')
area_summary = area_summary.merge(other_summary, on='urban_change', how='outer')
area_summary.fillna(0, inplace=True)

# Round and compute totals
area_summary[['slum_area_ha', 'buffer_area_ha', 'other_area_ha']] = area_summary[[
    'slum_area_ha', 'buffer_area_ha', 'other_area_ha'
]].round(2)
area_summary['total_ha'] = area_summary[['slum_area_ha', 'buffer_area_ha', 'other_area_ha']].sum(axis=1)

# Compute percentage distribution
area_summary['slum_%'] = (area_summary['slum_area_ha'] / area_summary['total_ha'] * 100).round(2)
area_summary['buffer_%'] = (area_summary['buffer_area_ha'] / area_summary['total_ha'] * 100).round(2)
area_summary['other_%'] = (area_summary['other_area_ha'] / area_summary['total_ha'] * 100).round(2)

In [ ]:
# styled table of summary
styled_table = (
    area_summary
    .style
    .format({
        'slum_area_ha': '{:.1f}',
        'buffer_area_ha': '{:.1f}',
        'other_area_ha': '{:.1f}',
        'total_ha': '{:.1f}',
        'slum_%': '{:.1f}%',
        'buffer_%': '{:.1f}%',
        'other_%': '{:.1f}%'
    })
    .set_caption("Urban Change Area Distribution by Zone (in Hectares and Percentages)")
    .hide(axis="index")
)
styled_table

## Section 2.5 Bar plot for area % with annotations

In [ ]:
import numpy as np

labels = area_summary['urban_change']
slum_pct = area_summary['slum_%']
buffer_pct = area_summary['buffer_%']
other_pct = area_summary['other_%']
slum_ha = area_summary['slum_area_ha']
buffer_ha = area_summary['buffer_area_ha']
other_ha = area_summary['other_area_ha']

x = np.arange(len(labels))
width = 0.25

fig, ax = plt.subplots(figsize=(12, 6))

bars1 = ax.bar(x - width, slum_pct, width, label='In Slums')
bars2 = ax.bar(x, buffer_pct, width, label='In Buffer Ring')
bars3 = ax.bar(x + width, other_pct, width, label='In Other Regions')

# Add absolute area (ha) as annotation
def annotate(bars, values):
    for bar, value in zip(bars, values):
        height = bar.get_height()
        ax.annotate(f'{value:.0f} ha',
                    xy=(bar.get_x() + bar.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom', fontsize=9)

annotate(bars1, slum_ha)
annotate(bars2, buffer_ha)
annotate(bars3, other_ha)

# Plot styling
ax.set_ylabel('Area Percentage (%)')
ax.set_title('Urban Change Category Distribution by Spatial Zone')
ax.set_xticks(x)
ax.set_xticklabels(labels, rotation=15)
ax.legend()
ax.yaxis.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
plt.tight_layout()
plt.show()